# Import libraries

In [1]:
import pandas as pd
from config import RAW_DIR, CLEAN_DIR
import numpy as np
import os

# Extract features and labels from the raw meta-analysis file

In [2]:
# load up our dataset
raw = pd.read_csv(os.path.join(RAW_DIR, "raw.csv"), index_col=[0, 3])
raw.iloc[:5, :20]

/Users/daniel/opt/anaconda3/envs/data/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10,11,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Author (year) BoiProject ID       RunID  \
Study_ID Sample Accession                                              
GMHI-23  SAMEA3879547      Vogtmann (2016)    PRJEB12449  ERR1293689   
         SAMEA3879551      Vogtmann (2016)    PRJEB12449  ERR1293705   
         SAMEA3879543      Vogtmann (2016)    PRJEB12449  ERR1293673   
         SAMEA3879565      Vogtmann (2016)    PRJEB12449  ERR1293761   
         SAMEA3879546      Vogtmann (2016)    PRJEB12449  ERR1293685   

                           Sequencing Platform Library layout (SE/PE)  \
Study_ID Sample Accession                                               
GMHI-23  SAMEA3879547      Illumina HiSeq 2000                 PAIRED   
         SAMEA3879551      Illumina HiSeq 2000                 PAIRED   
         SAMEA3879543      Illumina HiSeq 2000                 PAIRED   
         SAMEA3879565      Illumina HiSeq 2000                 PAIRED   
         SAMEA3879546      Illumina HiSeq 2000                 PAIRED   

                          Sample origin? (e.g., stool, mucosa) Disease status  \
Study_ID Sample Accession                                                       
GMHI-23  SAMEA3879547                                    stool        Control   
         SAMEA3879551                                    stool        Control   
         SAMEA3879543                                    stool        Control   
         SAMEA3879565                                    stool        Control   
         SAMEA3879546                                    stool        Control   

                                Sample_ID Subject Id (If available)  \
Study_ID Sample Accession                                             
GMHI-23  SAMEA3879547      MMRS84159866ST            MMRS84159866ST   
         SAMEA3879551      MMRS25211151ST            MMRS25211151ST   
         SAMEA3879543      MMRS32573774ST            MMRS32573774ST   
         SAMEA3879565      MMRS20257302ST            MMRS20257302ST   
         SAMEA3879546      MMRS86168210ST            MMRS86168210ST   

                          Age (Years)  Gender  \
Study_ID Sample Accession                       
GMHI-23  SAMEA3879547            67.0  Female   
         SAMEA3879551            60.0  Female   
         SAMEA3879543            82.0    Male   
         SAMEA3879565            62.0  Female   
         SAMEA3879546            79.0    Male   

                          Geographical Region or Population BMI (kg/m²)  \
Study_ID Sample Accession                                                 
GMHI-23  SAMEA3879547                                   USA    18.63574   
         SAMEA3879551                                   USA   20.466036   
         SAMEA3879543                                   USA   20.524438   
         SAMEA3879565                                   USA   20.665713   
         SAMEA3879546                                   USA   21.284602   

                          PHENOTYPE_Disease PHENOTYPE:Healthy_Nonhealthy  \
Study_ID Sample Accession                                                  
GMHI-23  SAMEA3879547               Healthy                      Healthy   
         SAMEA3879551               Healthy                      Healthy   
         SAMEA3879543               Healthy                      Healthy   
         SAMEA3879565               Healthy                      Healthy   
         SAMEA3879546               Healthy                      Healthy   

                             UNKNOWN  k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                                       
GMHI-23  SAMEA3879547      56.451568    0.861124                     0.861124   
         SAMEA3879551      39.905873    0.000000                     0.000000   
         SAMEA3879543      53.274915    0.151996                     0.151996   
         SAMEA3879565      59.044005    0.362985                     0.362985   
         SAMEA3879546      75.424715    0.001181         

In [3]:
# Select the taxonomic features
taxonomic_features = raw.iloc[:, 15:]

# Select the health status labels 
is_healthy = raw[["PHENOTYPE:Healthy_Nonhealthy"]] == "Healthy"

# Select the full phenotype labels
phenotype = raw[["PHENOTYPE_Disease"]]

# Remove bad samples 

In [4]:
non_illumina = [
    "454 GS FLX Titanium",
    "Ion Torrent PGM",
    "Ion Torrent Proton",
    "BGISEQ-500",
]

In [5]:
# Find the indices of samples we would like to remove

remove = (
    
    # we don't need a biomarker for obesity/underweight
    (phenotype == "Underweight").values.flatten() | 
    (phenotype == "Overweight").values.flatten() |
    (phenotype == "Obesity").values.flatten() |
    (phenotype == "Obese").values.flatten() |
    
    # to reduce batch effects, remove samples sequenced with non-illumina machines
    raw["Sequencing Platform"].isin(non_illumina).values.flatten() |
    
    # all of these studies have some issue
    (phenotype.index.get_level_values(0) == "P4") | # P4 treats the poop for extracting viral DNA
    (phenotype.index.get_level_values(0) == "P86") | # Healthy at baseline but half develop T2D 
    (phenotype.index.get_level_values(0) == "GMHI-19") | # Outlier study
    ((phenotype.index.get_level_values(0) == "P48") & (phenotype == "Healthy").values.flatten()) | # Alcohol or smoking
    (phenotype.index.get_level_values(0) == "P59") | # Are all technically healthy, but half are in heavily urbanized areas
    # and "Microbes with higher relative abundance in Chinese urban samples have been associated with disease in other studies"
    (phenotype.index.get_level_values(0) == "P63") # Deals with semisupercentenarians, i.e., 105 to 109 years old
)

In [6]:
# remove bad samples
taxonomic_features = taxonomic_features.iloc[~remove, :]
is_healthy = is_healthy.iloc[~remove, :]
phenotype = phenotype.iloc[~remove, :]

In [7]:
# normalize relative abundances
taxonomic_features = taxonomic_features.divide((100 - taxonomic_features["UNKNOWN"]), axis="rows")

In [8]:
# we don't need unknown column anymore
taxonomic_features = taxonomic_features.drop(labels=["UNKNOWN"], axis=1)

print(f"Our dataset has {taxonomic_features.shape[0]} samples with {taxonomic_features.shape[1]} unique taxnomic features")
print(f"Our dataset has {len(np.unique(taxonomic_features.index.get_level_values(0)))} independent studies")


Our dataset has 9045 samples with 3200 unique taxnomic features
Our dataset has 61 independent studies


In [9]:
# there are 7 taxonomic ranks, each one should sum to 1, so sum of each sample should be 7 (w/ floating point error)
taxonomic_features.sum(axis=1)

Study_ID  Sample Accession
GMHI-23   SAMEA3879547        6.998069
          SAMEA3879551        6.999311
          SAMEA3879543        6.996683
          SAMEA3879565        6.996989
          SAMEA3879546        6.975655
                                ...   
P140      SAMN07509557        7.000000
          SAMN07509558        1.000000
          SAMN07509546        6.999737
          SAMN07509552        7.000000
          SAMN07509921        6.998958
Length: 9045, dtype: float64

# Standardize names for diseased phenotypes

In [39]:
np.unique(phenotype)

array(['Advanced Dementia', 'Ankylosing spondylitis',
       'Atherosclerotic cardiovascular disease', 'Behcet’s disease',
       'Breast Cancer', 'Colorectal adenoma', 'Colorectal cancer',
       "Crohn's Disease", 'End-stage renal disease', 'Graves’ disease',
       'Healthy', 'Hypertension', 'Impaired glucose tolerance',
       'Liver Cirrhosis', 'NAFLD', 'Pancreatic cancer',
       'Rheumatoid arthritis', 'Schizophrenia', 'Type 2 diabetes',
       'Ulcerative colitis'], dtype=object)

In [40]:
dic = {
    "Adenoma (MP)" : "Colorectal adenoma",
    "Large adenoma" : "Colorectal adenoma",
    "Small adenoma" : "Colorectal adenoma",
    "adenoma" : "Colorectal adenoma",
    "advanced adenoma" : "Colorectal adenoma",
    "carcinoma" : "Colorectal adenoma",
    
    "Cancer" : "Colorectal cancer",
    "CRC" : "Colorectal cancer",
    
    "Crohn''s disease" : "Crohn's Disease",
    "CD" : "Crohn's Disease",
    "Crohns disease" : "Crohn's Disease",
    
    "Rheumatoid Arthritis" : "Rheumatoid arthritis",
    
    "IGT" : "Impaired glucose tolerance",
    
    "T2D" : "Type 2 diabetes",
    
    "schizophrenia" : "Schizophrenia",
    
    "ulcerative colitis" : "Ulcerative colitis",
    
    "ACVD" : "Atherosclerotic cardiovascular disease",
    
    "End-stage renal disease (ESRD)" : "End-stage renal disease",
    
    "Hypertension (HTN)" : "Hypertension",
    
    "ankylosing spondylitis" : "Ankylosing spondylitis",
    
}

phenotype = phenotype.replace({"PHENOTYPE_Disease" : dic}) 

In [43]:
np.unique(phenotype)

array(['Advanced Dementia', 'Ankylosing spondylitis',
       'Atherosclerotic cardiovascular disease', 'Behcet’s disease',
       'Breast Cancer', 'Colorectal adenoma', 'Colorectal cancer',
       "Crohn's Disease", 'End-stage renal disease', 'Graves’ disease',
       'Healthy', 'Hypertension', 'Impaired glucose tolerance',
       'Liver Cirrhosis', 'NAFLD', 'Pancreatic cancer',
       'Rheumatoid arthritis', 'Schizophrenia', 'Type 2 diabetes',
       'Ulcerative colitis'], dtype=object)

# Yay we are done, export stuff

In [45]:
taxonomic_features.to_csv(os.path.join(CLEAN_DIR, "taxonomic_features.csv"))
is_healthy.to_csv(os.path.join(CLEAN_DIR, "is_healthy.csv"))
phenotype.to_csv(os.path.join(CLEAN_DIR, "phenotype.csv"))

# Extract features and labels from the raw validation file

In [11]:
raw_cv = pd.read_csv(os.path.join(RAW_DIR, "raw_validation.csv"), index_col=[0, 12])
raw_cv.iloc[:5, :20]

Author (year) BoiProject ID Sample Accession  \
Study_ID Subject_ID                                                  
P65      ERAS10      Palleja (2018)    PRJEB20800   SAMEA104062441   
         ERAS10      Palleja (2018)    PRJEB20800   SAMEA104062442   
         ERAS10      Palleja (2018)    PRJEB20800   SAMEA104062443   
         ERAS10      Palleja (2018)    PRJEB20800   SAMEA104062444   
         ERAS11      Palleja (2018)    PRJEB20800   SAMEA104062445   

                          RunID  Sequencing Platform Library layout (SE/PE)  \
Study_ID Subject_ID                                                           
P65      ERAS10      ERR1995211  Illumina HiSeq 2000                 PAIRED   
         ERAS10      ERR1995212  Illumina HiSeq 2000                 PAIRED   
         ERAS10      ERR1995213  Illumina HiSeq 2000                 PAIRED   
         ERAS10      ERR1995214  Illumina HiSeq 2000                 PAIRED   
         ERAS11      ERR1995215  Illumina HiSeq 2000                 PAIRED   

                    Sample origin? (e.g., stool, mucosa)      Sample_ID  \
Study_ID Subject_ID                                                       
P65      ERAS10                                    Stool    ERAS10_Dag0   
         ERAS10                                    Stool  ERAS10_Dag180   
         ERAS10                                    Stool   ERAS10_Dag42   
         ERAS10                                    Stool    ERAS10_Dag8   
         ERAS11                                    Stool    ERAS11_Dag0   

                     Age (Years) Gender BMI (kg/m²) Health status Time point  \
Study_ID Subject_ID                                                            
P65      ERAS10              NaN    NaN         NaN       Healthy         D0   
         ERAS10              NaN    NaN         NaN       Healthy       D180   
         ERAS10              NaN    NaN         NaN       Healthy        D42   
         ERAS10              NaN    NaN         NaN       Healthy         D8   
         ERAS11              NaN    NaN         NaN       Healthy         D0   

                     clade_name   UNKNOWN  k__Archaea  \
Study_ID Subject_ID                                     
P65      ERAS10      ERR1995211  55.91622    0.000000   
         ERAS10      ERR1995212  31.85643    0.000000   
         ERAS10      ERR1995213  35.34897    0.000000   
         ERAS10      ERR1995214  57.00076    0.000000   
         ERAS11      ERR1995215  75.94112    0.662822   

                     k__Archaea|p__Euryarchaeota  \
Study_ID Subject_ID                                
P65      ERAS10                         0.000000   
         ERAS10                         0.000000   
         ERAS10                         0.000000   
         ERAS10                         0.000000   
         ERAS11                         0.662822   

                     k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Subject_ID                                                   
P65      ERAS10                                            0.000000   
         ERAS10                                            0.000000   
         ERAS10                                            0.000000   
         ERAS10                                            0.000000   
         ERAS11                                            0.662822   

                     k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Subject_ID                                                                         
P65      ERAS10                                               0.000000                      
         ERAS10                                               0.000000                      
         ERAS10                                               0.000000                      
         ERAS10                                               0.000000                      
         ERAS11                                               0.66282

In [12]:
# Select the taxonomic features
taxonomic_features_cv = raw_cv.iloc[:, 14:]

# Select the full phenotype labels
phenotype_cv = raw_cv[["Health status"]]

# Select the health status labels 
is_healthy_cv = phenotype_cv == "Healthy"

# Select time point info 
time_point_cv = raw_cv[["Time point"]]

In [13]:
# normalize relative abundances
taxonomic_features_cv = taxonomic_features_cv.divide((100 - taxonomic_features_cv["UNKNOWN"]), axis="rows")

In [14]:
# we don't need unknown column anymore
taxonomic_features_cv = taxonomic_features_cv.drop(labels=["UNKNOWN"], axis=1)

In [15]:
# there are 7 taxonomic ranks, each one should sum to 1, so sum of each sample should be 7 (w/ floating point error)
taxonomic_features_cv.sum(axis=1)

Study_ID  Subject_ID
P65       ERAS10        6.999077
          ERAS10        6.999857
          ERAS10        7.000002
          ERAS10        6.991888
          ERAS11        6.996988
                          ...   
P116      22            6.999367
          23            6.999259
          24            6.998883
          63            6.999739
          31            6.996203
Length: 691, dtype: float64

# Yay we are done, export stuff

In [16]:
taxonomic_features_cv.to_csv(os.path.join(CLEAN_DIR, "taxonomic_features_cv.csv"))
phenotype_cv.to_csv(os.path.join(CLEAN_DIR, "phenotype_cv.csv"))
is_healthy_cv.to_csv(os.path.join(CLEAN_DIR, "is_healthy_cv.csv"))
time_point_cv.to_csv(os.path.join(CLEAN_DIR, "time_point_cv.csv"))